# Temperaturas en ciudades de Bolivia
*Una exploración visual con datos de baja resolución de SENAMHI*

In [6]:
# Dependencias

import pandas as pd
import matplotlib.pyplot as plt
import csv
import re
import datetime as dt
from matplotlib import cm, ticker, colors
import numpy as np
from scipy.interpolate import interp1d
from IPython.display import Image, display
import math

In [7]:
# Algunas definiciones de estilo para matplotlib

plt.style.use('resources/clean.mplstyle')

SENAMHI produce datos de temperatura a nivel de ciudad y el INE los consolida y publica mensualmente. Una tabla en excel puede ser descargada [desde el portal del INE](https://www.ine.gob.bo/index.php/medio-ambiente/clima-y-atmosfera/).  

In [8]:
# El código en este cuaderno *debería* funcionar correctamente con nuevas actualizaciones en este documento. 
EXCEL_INE = 'resources/BOLIVIA - TEMPERATURA MEDIA POR CIUDADES, SEGÚN AÑO Y MES, 1990 - 2023.xlsx'

In [9]:
# Definiciones para meses que nos ayudarán a leer meses y producir etiquetas

meses = {
        'Enero': 1,
        'Febrero': 2,
        'Marzo': 3,
        'Abril': 4,
        'Mayo': 5,
        'Junio': 6,
        'Julio': 7,
        'Agosto': 8,
        'Septiembre': 9,
        'Octubre': 10,
        'Noviembre': 11,
        'Diciembre': 12
    }

meses_backwards = {meses[m]:m for m in meses.keys()}

Leemos este documento y ponemos la información en una forma cómoda para futuras operaciones. Estos registros son promedios mensuales de mediciones en grados centígrados para ciudades capitales más El Alto.

In [10]:
def parse_ine():
    """
    Selecciona los campos relevantes de `EXCEL_INE` y pone en forma fechas y tipos de valores.
    Es una función particularmente sensible al formato que utiliza el INE, que es súper idiosincrático
    y probablemente cambie en los próximos años (meses?).
    """
    
    excel = pd.read_excel(EXCEL_INE, skiprows=8, skipfooter=5)
    excel = excel.drop(columns=[col for col in excel.columns if 'Unnamed' in col])
    periodo_col = excel.columns[0]
    excel = excel[excel[periodo_col].notna()]
    
    header = ['AÑO'] + list(excel.columns)
    entries = []
    for i, row in excel.iterrows():
        if len(re.findall('[0-9]+', str(row[periodo_col]))) > 0:
            anio = row[periodo_col]
        else:
            entries.append([anio] + list(row))
            
    ine = pd.DataFrame(entries, columns=header)
    ine.columns = [col.strip().lower().replace(' ', '_') for col in ine.columns]
    ine.año = ine.año.astype(str).apply(lambda _: re.findall('[0-9]+', _)[0]).astype(int)
    ine.insert(1, 'mes', ine.periodo.apply(lambda _: meses[_.strip()]))
    ine.drop(columns=['periodo'], inplace=True)
    for col in ine.columns :
        if col not in ['año', 'mes']:
            ine[col] = ine[col].replace('n.d.', None).astype(float)
    ine.insert(0, 'fecha', ine[['año', 'mes']].apply(lambda _: dt.datetime(_['año'], _['mes'], 1), axis=1))

    return ine

ine = parse_ine()
ine

,fecha,año,mes,sucre,la_paz,cochabamba,oruro,potosí,tarija,santa_cruz,trinidad,cobija,el_alto
0,1990-01-01,1990,1,15.900000,11.600000,19.196000,11.600000,8.000000,20.800000,25.850000,26.000000,25.450000,8.008000
1,1990-02-01,1990,2,15.050000,11.700000,18.316000,10.950000,7.400000,19.800000,26.400000,27.400000,25.650000,8.307400
2,1990-03-01,1990,3,16.450000,12.100000,19.629000,10.250000,7.800000,21.200000,26.750000,26.900000,26.150000,8.764500
3,1990-04-01,1990,4,16.250000,11.800000,18.215000,9.100000,8.000000,20.400000,25.550000,26.400000,25.950000,8.520000
4,1990-05-01,1990,5,14.100000,10.700000,16.201000,6.050000,5.800000,16.200000,21.450000,23.400000,24.750000,7.114500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,2023-06-01,2023,6,11.739931,11.621667,14.671667,4.250000,7.341379,13.943500,21.996667,23.793833,25.345000,5.078333
402,2023-07-01,2023,7,13.288710,12.674194,16.191935,3.980645,5.282258,14.490323,21.851613,25.272581,26.796774,5.080645
403,2023-08-01,2023,8,14.098077,12.277419,17.235484,6.304348,6.374074,16.732258,25.916129,26.404839,28.364516,5.819355
404,2023-09-01,2023,9,16.026786,13.917241,20.548214,10.618750,11.246667,19.934783,27.269643,28.922222,29.683929,8.344643


*Cómo cambia la temperatura en la ciudad a través de los años?*

Primero una vista simple que ayude a comparar meses de distintos años en una misma ciudad. La escala de colores está asociada al orden de años entre 1990 y 2023, con énfasis en 2023. El objetivo es ganar una primera intuición, con unidades fáciles de interpretar, sobre la estacionalidad anual de temperaturas, la dirección de cambios durante este periodo y la particularidad de 2023. 

In [11]:
def linemonths_ine(dep, dataframe=ine, cmap='RdBu_r', fn_prefix='ine_comparacion_mensual_', subtitle_prefix='Temperatura media por mes (°C)', footer='Fuente: SENAMHI via INE', y_pivot=None):
    """
    Produce un gráfico de líneas para una serie de temperaturas en una ciudad.
    Como todo buen código para matplotlib, es innecesariamente verboso y feo para leer.
    Algunos parámetros:
    - dep: el nombre de la ciudad
    - dataframe: la tabla, por defecto `ine`
    - cmap: la escala de colores en el inventario de matplotlib
    - fn_prefix: el prefijo del nombre de file
    - subtitle_prefix: el prefijo del subtítulo
    - footer: el texto al pie de la gráfica
    - y_pivot: la posición en el eje vertical para dibujar una línea de referencia
    """
    
    
    colormap_max = .9
    dfi = dataframe.pivot_table(index='mes', columns='año', values=dep).copy()
    colormap = [plt.get_cmap(cmap)(i) for i in np.linspace(0,colormap_max,len(dfi.columns))]
    highlight = dfi.columns[-1]
    fn = f'plots/{fn_prefix}{dep}.png'

    f, ax = plt.subplots(1,1,figsize=(15,4))

    for año, color in zip(dfi.columns, colormap):

        dfi_y = dfi[año].dropna()
        x_curve = np.linspace(dfi_y.index.min(), dfi_y.index.max(), 1000)
        y_curve = interp1d(x=dfi_y.dropna().index, y=dfi_y.interpolate(), kind=2)(x_curve)

        ax.plot(
            x_curve,
            y_curve, 
            color=color if año != highlight else plt.get_cmap(cmap)(1.), 
            lw=.9 if año != highlight else 1.5, 
            alpha=.5 if año != highlight else .9
        )
        
        ax.scatter(
            dfi.index, 
            dfi[año], 
            color=color if año != highlight else plt.get_cmap(cmap)(1.), 
            s=3, 
            alpha=.9
        )

    ax.grid()
    ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
    ax.xaxis.set_major_formatter(
        ticker.FuncFormatter(lambda _, pos: meses_backwards[int(_)] if 12 > _ > 1 else '')
    )
    y_domain = [dfi.min().min(), dfi.max().max()]
    y_padding = (y_domain[1] - y_domain[0]) * .03
    ax.set_ylim(y_domain[0] - y_padding, y_domain[1] + y_padding)
    ax.set_xlim(.99, 12.01)
    
    if y_pivot != None:
        ax.axhline(y_pivot, linestyle=':', color='#000', alpha=.8, linewidth=.9)
    
    cbar = f.colorbar(
        cm.ScalarMappable(
            norm=colors.Normalize(dfi.columns[0], highlight), 
            cmap=colors.ListedColormap(plt.get_cmap(cmap)(np.linspace(0,colormap_max,1000))).with_extremes(over=plt.get_cmap(cmap)(1.))
        ),
        ax=ax, 
        pad=.02,
        shrink=.7,
        alpha=.9,
        drawedges=False,
        orientation='vertical',
        ticks=list(range(1995,2025,5)),
        extend='max',
        extendrect=True,
        extendfrac=.08
    )
    cbar.outline.set_alpha(.1)
    
    ax.annotate(
        f'{subtitle_prefix} desde 1990 hasta {meses_backwards[dfi[highlight].dropna().index[-1]]} de {highlight}',
        xycoords='axes fraction',
        xy=(.0,1.075),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=10,
        fontweight='light',
        alpha=.7
    )
    
    ax.annotate(
        dep.replace('_',' ').upper(),
        xycoords='axes fraction',
        xy=(.0,1.1),
        ha='left',
        va='bottom',
        fontfamily='Archivo',
        fontsize=17,
        fontweight='bold',
        alpha=.7
    )
    
    ax.annotate(
        footer,
        xycoords='axes fraction',
        xy=(.0,-.13),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=6,
        fontweight='light',
        alpha=.7
    )
    
    f.savefig(
        fn,
        pad_inches=.3,
        bbox_inches='tight'
    )
    
    plt.close()
    display(
        Image(url=fn, embed=False)
    )

In [12]:
# columnas con nombres de ciudades (por qué `deps` ...¿?)
deps = ine.columns[3:]

In [13]:
# un gráfico para cada ciudad
for dep in deps:
    linemonths_ine(dep)

Es claro que años más recientes registran temperaturas más altas y que 2023, particularmente luego de junio, representa un salto importante. Para comprender mejor cuánto crecen temperaturas sería útil tener un punto de referencia, digamos el promedio mensual de temperaturas entre 1990 y 2000. 

In [14]:
baseline = ine[(ine.fecha > '1990-01-01') & (ine.fecha < '2000-01-01')].groupby('mes')[deps].mean()
baseline

,sucre,la_paz,cochabamba,oruro,potosí,tarija,santa_cruz,trinidad,cobija,el_alto
mes,,,,,,,,,,
1,15.766667,12.477778,19.380778,11.994444,9.328571,20.844444,26.716667,26.988889,25.927778,9.055756
2,15.340000,12.590000,18.755600,11.375000,9.125000,20.170000,26.075000,26.500000,25.830000,9.013810
3,15.110000,12.240000,18.879100,11.245000,8.371429,19.870000,26.240000,27.180000,25.960000,8.696890
4,14.920000,12.050000,18.306200,9.545000,7.962500,18.580000,24.405000,26.210000,25.550000,8.092320
5,13.795000,11.340000,16.073100,5.695000,6.200000,15.877778,22.420000,24.450000,24.835000,6.722540
6,12.605000,9.940000,13.704400,3.090000,4.325000,13.670000,20.920000,23.310000,23.690000,4.997240
7,12.125000,9.650000,13.903300,3.315000,4.522222,13.144444,20.330000,22.510000,23.365000,5.015490
8,13.225000,10.260000,15.660300,5.290000,6.000000,15.277778,22.535000,24.030000,24.175000,5.799120
9,14.460000,11.400000,17.603100,7.630000,7.266667,17.290000,24.290000,24.790000,25.760000,7.227890


Construimos una nueva tabla donde cada valor representa la diferencia entre la observación y el promedio en el mismo mes y ciudad durante el periodo de referencia. Esta tabla debería apuntar más fácilmente a dónde, cuándo y en qué dimensión ocurren anomalías en temperaturas.

In [15]:
ine_anom = []
for i, row in ine.iterrows():
    for dep in deps:
        row[dep] = row[dep] - baseline.loc[row['mes'], dep]
    ine_anom.append(row)

ine_anom = pd.DataFrame(ine_anom)
ine_anom

,fecha,año,mes,sucre,la_paz,cochabamba,oruro,potosí,tarija,santa_cruz,trinidad,cobija,el_alto
0,1990-01-01,1990,1,0.133333,-0.877778,-0.184778,-0.394444,-1.328571,-0.044444,-0.866667,-0.988889,-0.477778,-1.047756
1,1990-02-01,1990,2,-0.290000,-0.890000,-0.439600,-0.425000,-1.725000,-0.370000,0.325000,0.900000,-0.180000,-0.706410
2,1990-03-01,1990,3,1.340000,-0.140000,0.749900,-0.995000,-0.571429,1.330000,0.510000,-0.280000,0.190000,0.067610
3,1990-04-01,1990,4,1.330000,-0.250000,-0.091200,-0.445000,0.037500,1.820000,1.145000,0.190000,0.400000,0.427680
4,1990-05-01,1990,5,0.305000,-0.640000,0.127900,0.355000,-0.400000,0.322222,-0.970000,-1.050000,-0.085000,0.391960
...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,2023-06-01,2023,6,-0.865069,1.681667,0.967267,1.160000,3.016379,0.273500,1.076667,0.483833,1.655000,0.081093
402,2023-07-01,2023,7,1.163710,3.024194,2.288635,0.665645,0.760036,1.345878,1.521613,2.762581,3.431774,0.065155
403,2023-08-01,2023,8,0.873077,2.017419,1.575184,1.014348,0.374074,1.454480,3.381129,2.374839,4.189516,0.020235
404,2023-09-01,2023,9,1.566786,2.517241,2.945114,2.988750,3.980000,2.644783,2.979643,4.132222,3.923929,1.116753


Comparamos visualmente estos valores en la misma lógica de los primeros gráficos, donde cada valor representa la diferencia entre la temperatura observada en un mes y ciudad, y la temperatura promedio en el periodo de referencia.

In [16]:
for dep in deps:
    linemonths_ine(
        dep, 
        dataframe=ine_anom, 
        fn_prefix='ine_anomalias_', 
        subtitle_prefix='Anomalías mensuales de temperatura (°C)', 
        footer='Fuente: SENAMHI via INE. Periodo de referencia: 1990 a 2000',
        y_pivot=0
    )

La tendencia a mayores temperaturas en años más recientes es clara en la mayoría de las ciudades, y las temperaturas en 2023, particularmente luego de junio, destacan en todos los casos. *Cómo se comparan anomalías en un mismo año, digamos 2023, entre distintas ciudades?* Realizamos una composición que reuna líneas de todas las ciudades para un mismo año. Asignamos colores a cada ciudad siguiendo el orden de los promedios de anomalías en cada ciudad y año, donde ciudades con mayores anomalías respecto al periodo de referencia aparecen más claras y brillantes, y ciudades con menos diferencias se muestran más opacas.

In [18]:
def plot_compare_anom(año=2023, cmap='magma'):
    """
    Dibuja líneas de anomalías durante un año para todas las ciudades.
    El orden de ciudades según su promedio de anomalías determina atributos de cada línea
    como su color, opacidad y grosor. La etiqueta hace explícito este orden. 
    Mucho código son improvisaciones para manejar situaciones como líneas con campos vacíos o
    la posición de la etiqueta.
    """
    
    def plot_curve(x, y, interp, label=None):
        x_curve = np.linspace(x.min(), x.max(), 1000)
        y_curve = interp1d(x=x, y=y, kind=interp)(x_curve)
        ax.plot(
            x_curve,
            y_curve,
            color=color,
            alpha=alpha,
            lw=(i + 3) ** .4,
            label=label
        )

    dfi = ine_anom.loc[ine_anom.año == año, list(deps) + ['mes']].copy().set_index('mes')
    # order = dfi.iloc[-1].sort_values().index
    order = dfi.mean().sort_values().index

    colormap = [plt.get_cmap(cmap)(i) for i in np.linspace(0,.85,len(order))]
    alphas = [(i + 1) / 10 for i in range(0,len(order))]
    
    f, ax = plt.subplots(1,1,figsize=(len(dfi), 4))

    for i, (dep, color, alpha) in enumerate(zip(order, colormap, alphas)):
        
        labelled = False
        label = ' '.join([i[0].upper() + i[1:] for i in dep.split('_')])
        
        if dfi[dep].isna().sum() > 0:
        
            lines = []
            line = []
            for x, y in dfi[dep].items():
                if not math.isnan(y):
                    line.append({'x':x, 'y':y})
                else:
                    if line:
                        lines.append(line)
                    line = []
            if line:
                lines.append(line)

            for line in lines:

                linedf = pd.DataFrame(line)
                plot_curve(linedf.x, linedf.y, 2 if len(linedf) > 2 else 'linear', None if labelled else label)
                labelled = True
                
        else:
            
            plot_curve(dfi.index, dfi[dep], 2, label)

    ax.grid()
    y_domain = [dfi.min().min(), dfi.max().max()]
    y_padding = (y_domain[1] - y_domain[0]) * .1
    ax.set_ylim(y_domain[0] - y_padding, y_domain[1] + y_padding)
    ax.set_xlim(.8,len(dfi) + .2)

    ax.xaxis.set_major_locator(ticker.MaxNLocator(len(dfi)))
    ax.xaxis.set_major_formatter(
        ticker.FuncFormatter(lambda _, pos: meses_backwards[int(_)] if len(dfi) > _ > 1 else '')
    )

    ax.axhline(0, xmin=.02, xmax=.98, linestyle=':', color='#000', alpha=.8, linewidth=.9)

    legend = ax.legend(
        loc='center right',
        bbox_to_anchor=(1 + (1.35 / len(dfi)), .5),
        labelcolor=list(reversed(colormap)),
        handletextpad=.3,
        reverse=True,
    )

    for label, alpha in zip(legend.get_texts(), reversed(alphas)):
        label.set_alpha(alpha)
        label.set_fontweight('bold')

    for line in legend.get_lines():
        line.set_marker('o')
        line.set_linestyle(' ')
    
    
    ax.annotate(
        'Anomalías mensuales de temperatura (°C) en ciudades',
        xycoords='axes fraction',
        xy=(.0,1.075),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=10,
        fontweight='light',
        alpha=.7
    )
    
    ax.annotate(
        año,
        xycoords='axes fraction',
        xy=(.0,1.1),
        ha='left',
        va='bottom',
        fontfamily='Archivo',
        fontsize=17,
        fontweight='bold',
        alpha=.7
    )
    
    ax.annotate(
        'Fuente: SENAMHI via INE. Periodo de referencia: 1990 a 2000',
        xycoords='axes fraction',
        xy=(.0,-.13),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=6,
        fontweight='light',
        alpha=.7
    )
    
    fn = f'plots/ine_anomalias_{año}.png'
    
    f.savefig(
        fn,
        pad_inches=.3,
        bbox_inches='tight'
    )
    
    plt.close()
    display(
        Image(url=fn, embed=False)
    )

Comparemos anomalías mensuales en ciudades entre 2018 y 2023.

In [19]:
for año in reversed(range(2018, 2024)):
    plot_compare_anom(año)

Parece existir un patrón en el orden de ciudades donde observamos mayores temperaturas respecto al periodo de referencia. *Cuán consistente es este patrón desde 1990?* Sería útil comparar anomalías entre ciudades a través de todos los años. Construimos una nueva tabla de anomalías con el mismo periodo de referencia, pero esta vez a escala anual. De esta manera podemos poner de fondo variaciones entre estaciones y observar la tendencia general de temperaturas en los últimos 30 años.

In [20]:
baseline_general = ine[(ine.fecha > '1990-01-01') & (ine.fecha < '2000-01-01')][deps].mean()
baseline_general

sucre         14.592773
la_paz        11.703361
cochabamba    17.646042
oruro          8.487815
potosí         7.583838
tarija        17.988696
santa_cruz    24.366807
trinidad      25.533051
cobija        25.291597
el_alto        7.604903
dtype: float64

In [21]:
ine_anom_anual = ine.groupby('año')[deps].mean().sub(baseline_general)
ine_anom_anual

,sucre,la_paz,cochabamba,oruro,potosí,tarija,santa_cruz,trinidad,cobija,el_alto
año,,,,,,,,,,
1990,0.257227,-0.695028,-0.143459,-0.241982,-0.675505,0.177971,-0.316807,0.258616,-0.145763,-0.146644
1991,0.682227,-0.878361,0.129874,-0.654482,0.457828,0.274941,-0.062640,-0.316384,0.029237,-0.051628
1992,0.236394,-0.603361,-0.150126,-0.671148,-1.256566,-0.438696,-0.904307,0.183616,-0.004097,-0.023761
1993,0.236394,-0.411695,-0.347770,-0.129482,NaN,0.189082,-0.308473,-0.796687,-0.320763,0.070231
1994,0.144727,-0.236695,0.152291,0.124685,-0.475505,0.402971,0.658193,-0.491384,0.358403,0.213239
1995,0.298894,0.304972,-0.162792,0.249685,1.266162,0.119638,0.212360,0.575282,0.650070,0.510714
1996,-0.763606,-0.195028,-0.097376,0.003852,0.088889,-0.372029,-0.462640,0.300282,-0.099930,-0.232011
1997,-0.471940,0.396639,-0.096709,-0.054482,0.249495,0.719638,0.933193,0.025282,0.754237,-0.346744
1998,0.348894,1.696639,0.820624,1.145518,0.641162,0.094638,0.408193,0.175282,-0.341597,0.511764


Y creamos una composición que reune anomalías anuales para cada ciudad donde, como en los gráficos anteriores, los colores son asignados según el promedio de anomalías para cada ciudad, donde ciudades con mayores cambios son representadas con colores más cálidos y brillantes.

In [22]:
def plot_anual_compare(cmap='magma'):
    """
    Esta función es una modificación mínima de la anterior y probablemente 
    debería ser implementada como una extensión con un par de parámetros adicionales.
    """
    
    def plot_curve(x, y, interp, label=None):
        x_curve = np.linspace(x.min(), x.max(), 1000)
        y_curve = interp1d(x=x, y=y, kind=interp)(x_curve)
        ax.plot(
            x_curve,
            y_curve,
            color=color,
            alpha=alpha,
            lw=(i + 3) ** .4,
            label=label
        )

    dfi = ine_anom_anual.copy()
    order = dfi.mean().sort_values().index

    colormap = [plt.get_cmap(cmap)(i) for i in np.linspace(0,.85,len(order))]
    alphas = [(i + 1) / 10 for i in range(0,len(order))]
    
    f, ax = plt.subplots(1,1,figsize=(15, 6))

    for i, (dep, color, alpha) in enumerate(zip(order, colormap, alphas)):
        
        labelled = False
        label = dep
        
        if dfi[dep].isna().sum() > 0:
        
            lines = []
            line = []
            for x, y in dfi[dep].items():
                if not math.isnan(y):
                    line.append({'x':x, 'y':y})
                else:
                    if line:
                        lines.append(line)
                    line = []
            if line:
                lines.append(line)

            for line in lines:

                linedf = pd.DataFrame(line)
                plot_curve(linedf.x, linedf.y, 2 if len(linedf) > 2 else 'linear', None if labelled else label)
                labelled = True
                
        else:
            
            plot_curve(dfi.index, dfi[dep], 2, label)

    ax.grid()
    y_domain = [dfi.min().min(), dfi.max().max()]
    y_padding = (y_domain[1] - y_domain[0]) * .1
    ax.set_ylim(y_domain[0] - y_padding, y_domain[1] + y_padding)
    ax.set_xlim(dfi.index.min() - .2, dfi.index.max() + .2)

    ax.xaxis.set_major_locator(ticker.MaxNLocator(len(dfi)))

    ax.axhline(0, xmin=.02, xmax=.98, linestyle=':', color='#000', alpha=.8, linewidth=.9)

    legend = ax.legend(
        loc='center right',
        bbox_to_anchor=(1 + (3 / len(dfi)), .5),
        labelcolor=list(reversed(colormap)),
        handletextpad=.3,
        reverse=True,
    )

    for label, alpha in zip(legend.get_texts(), reversed(alphas)):
        label.set_alpha(alpha)
        label.set_fontweight('bold')

    for line in legend.get_lines():
        line.set_marker('o')
        line.set_linestyle(' ')
    
    
    ax.annotate(
        'Comparada con el promedio entre 1990 y 2000',
        xycoords='axes fraction',
        xy=(.0,1.075),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=10,
        fontweight='light',
        alpha=.7
    )
    
    ax.annotate(
        'Temperatura anual media (°C)',
        xycoords='axes fraction',
        xy=(.0,1.1),
        ha='left',
        va='bottom',
        fontfamily='Archivo',
        fontsize=17,
        fontweight='bold',
        alpha=.7
    )
    
    ax.annotate(
        'Fuente: SENAMHI via INE.',
        xycoords='axes fraction',
        xy=(.0,-.1),
        ha='left',
        va='top',
        fontfamily='Archivo',
        fontsize=6,
        fontweight='light',
        alpha=.7
    )
    
    fn = f'plots/ine_anomalias_anuales.png'
    
    f.savefig(
        fn,
        pad_inches=.3,
        bbox_inches='tight'
    )
    
    plt.close()
    display(
        Image(url=fn, embed=False)
    )
    
plot_anual_compare()

El orden de ciudades según sus anomalías de temperatura desde 1990 es similar al observado en los años más recientes. Ciudades más al nor-oeste del país experimentan un mayor distanciamiento del periodo de referencia.